In [1]:
import json
import requests
from IPython.display import Image
import os
from dotenv import load_dotenv
import pandas as pd
from pymongo import MongoClient
import math

#Install first
#from pprint import pprint 

In [2]:
load_dotenv()

url = 'https://api.foursquare.com/v2/venues/explore'
tokenID = os.getenv("CLIENT_ID")
tokenSCRT = os.getenv("CLIENT_SECRET")

params = dict(
client_id=tokenID,
client_secret=tokenSCRT,
v='20180323',
ll='40.7243,-74.0018',
#query='Starbucks',
limit=2,
radius=100,
section="food"
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)



In [3]:
data['response']

{'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
 'headerLocation': 'SoHo',
 'headerFullLocation': 'SoHo, New York',
 'headerLocationGranularity': 'neighborhood',
 'query': 'food',
 'totalResults': 4,
 'suggestedBounds': {'ne': {'lat': 40.7252000009, 'lng': -74.00061465809291},
  'sw': {'lat': 40.723399999099996, 'lng': -74.00298534190709}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
     'venue': {'id': '49d4eeaff964a520555c1fe3',
      'name': 'Boqueria',
      'contact': {},
      'location': {'address': '171 Spring St',
       'crossStreet': 'at Thompson St',
       'lat': 40.7248,
       'lng': -74.00222,
       'labeledLatLngs': [{'label': 'display',
         'lat': 40.7248,
         'lng': -74.0022

In [4]:
#Funciones
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

#ny = geocode("New York")
#ny

# https://docs.mongodb.com/manual/reference/operator/query/near/
def withGeoQuery(location,maxDistance=10000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location if type(location)==dict else geocode(location),
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }
#query = withGeoQuery(ny)
#pprint(query)


def asGeoJSON(lat,lng):
    try:
        #Convierte a float (originalmente son numpy.float64)
        lat = float(lat)
        lng = float(lng)
        #Si el valor es NaN, convierte localización en None (null en lenguaje Mongodb)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        #print("Invalid data")
        return None
        



# Conexión con Mongodb y generación del GEOINDEX
---

In [5]:
#Mongo connection
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
companies = list(db["companies"].find({},{"name":1,"offices":1,"category_code":1}).limit(5))
df = pd.DataFrame(companies)
df.head(8)

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
4,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."


In [6]:
# 1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
df = df.explode('offices')
display(df.head(6))

#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
display(dfOfficeData.head(6))

#3. Concatenamos con el df original y limpiamos los datos
cleanData = pd.concat([df,dfOfficeData], axis=1)
cleanData = cleanData.drop(columns=["_id","offices"])
display(cleanData.head(6))


,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"{'description': 'Headquarters', 'address1': '4..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '..."
4,52cdef7c4bab8bd675297d90,Postini,web,"{'description': None, 'address1': '959 Skyway ..."


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
4,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


In [8]:
#Usando la funciónn asGeoJSON: Generamos la columna localización POINT(latitude,longitude)
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData = cleanData.drop(columns=["latitude","longitude"])
cleanData.head(8)

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
3,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."
